In [369]:
import csv
import pandas as pd
import numpy as np

In [370]:
target = 'scan/wheel-active-1.csv'

In [371]:
# open file
f = open('../' + target, 'r')
data = np.array(list(csv.reader(f)))
f.close()

In [372]:
# convert to dataframe
df = pd.DataFrame(data[:, 1:]).astype(np.float64)
df.index = data[:, 0]
df.columns = [
    'premium', 'roc', 'req_capital', 'be', 'be_moneyness', 
    'prob_be_delta', 'prob_be_iv', 'iv', 'iv_skew', 'drop_dist',
    'udl_year_ret', 'udl_year_ret_r2', 'udl_year_market_corr', 'udl_hist_vol', 
    'udl_iv_percentile', 'udl_hv_percentile', 'above_be_percentile'
]

In [377]:
# filter dataframe
df = df[df['roc'] >= 0.01]
df = df[df['prob_be_iv'] >= 0.75]
df = df[df['prob_be_delta'] >= 0.75]
df = df[df['be_moneyness'] <= 0.60]

In [378]:
def normalize(col):
    return (col - col.min()) / (col.max() - col.min())

# refine columns
df_filt = pd.DataFrame().astype(np.float64)
df_filt['premium'] = df['premium']
df_filt['break_even'] = df['be']
df_filt['be_moneyness'] = df['be_moneyness']
df_filt['drop_dist'] = df['drop_dist']
df_filt['roc'] = df['roc']
df_filt['prob_be_delta'] = df['prob_be_delta']
df_filt['prob_be_iv'] = df['prob_be_iv']
df_filt['ivp_hvp_diff'] = df['udl_iv_percentile'] - df['udl_hv_percentile']
df_filt['corr'] = df['udl_year_market_corr']
df_filt.index = df.index

In [387]:
# sort dataframe
df_filt = df_filt.sort_values('drop_dist', ascending=True)
df_filt_top = df_filt.nlargest(50, 'drop_dist')

In [388]:
# output results
pd.set_option('display.max_rows', 50)
df_filt_top

,premium,break_even,be_moneyness,drop_dist,roc,prob_be_delta,prob_be_iv,ivp_hvp_diff,corr
SRNE Nov 20 2020 $5.00 Put,25.0,4.75,0.40529,0.01144,0.05263,0.96118,0.80420,-0.27150,0.07967
NIO Nov 20 2020 $9.00 Put,15.0,8.85,0.41804,0.01119,0.01695,0.98121,0.89447,-0.44344,0.21817
TQQQ Nov 20 2020 $55.00 Put,86.0,54.14,0.41855,0.01118,0.01588,0.97846,0.94584,-0.00453,0.94576
ENPH Nov 20 2020 $35.00 Put,49.0,34.51,0.42443,0.01107,0.01420,0.96973,0.95433,-0.27149,0.50736
INO Nov 20 2020 $5.00 Put,27.0,4.73,0.42690,0.01102,0.05708,0.97427,0.81637,0.00905,-0.08681
CVNA Nov 20 2020 $100.00 Put,114.0,98.86,0.43653,0.01084,0.01153,0.98518,0.96483,0.31674,0.57146
NIO Nov 20 2020 $10.00 Put,22.0,9.78,0.46197,0.01035,0.02249,0.96879,0.87621,-0.44344,0.21817
SRNE Nov 20 2020 $6.00 Put,50.0,5.50,0.46928,0.01021,0.09091,0.92450,0.77773,-0.27150,0.07967
TNA Nov 20 2020 $15.00 Put,28.0,14.72,0.47808,0.01004,0.01902,0.96449,0.92627,-0.06335,0.91525
AAL Nov 20 2020 $6.00 Put,11.0,5.89,0.47808,0.01004,0.01868,0.97693,0.92873,-0.22624,0.45886
